# Bring data

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [70]:
#bring your data!
path = input('please type your path to the database: ')
data = pd.read_csv(f'{path}')
data.head()

#/home/lucaspancotto/code/JoacoSoulez/mental_health_first_aid_evaluation/data/clean_depressionvspossitive

please type your path to the database: /home/lucaspancotto/code/JoacoSoulez/mental_health_first_aid_evaluation/data/clean_depressionvspossitive


/tmp/ipykernel_24394/1042925164.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'{path}')


,Unnamed: 0,ids,Tweet,label
0,799999,1467822272,love health4uandpets u guys r best,0
1,800000,1467822273,im meeting one besties tonight cant wait girl ...,0
2,800001,1467822283,darealsunisakim thanks twitter add sunisa got ...,0
3,800002,1467822287,sick really cheap hurts much eat real food plu...,0
4,800003,1467822293,lovesbrooklyn2 effect everyone,0


In [71]:
positive_data = data[data.label == 0]


In [72]:
depression_data = data[data.label==1]

In [73]:
sample_positive_data = positive_data.sample( n  = 2345)

In [74]:
sample_data = sample_positive_data.append(depression_data)

/tmp/ipykernel_24394/2366412613.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_data = sample_positive_data.append(depression_data)


In [75]:
sample_data =  sample_data.sample(frac=1)

In [76]:
text_col = input('choose the text column of your data')
X = sample_data[f'{text_col}']

target_col = input('choose the target column of your data')
y = sample_data[f'{target_col}']

choose the text column of your dataTweet
choose the target column of your datalabel


# naive bayes model

## cross validation

In [103]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV , cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize

In [31]:
tfid = TfidfVectorizer(ngram_range=(4,5))
nb = MultinomialNB()

In [32]:
X = X.apply(str)
#X.apply(word_tokenize)
X

147717    way les paradise looking bk bridge togoski han...
802123    tropical depression gon na steal sun away mm f...
801293    smile cured depression emoji heavy red heart e...
800991    kudos michael phelps breaking stigma around ta...
110237    retrorewind pls play journey stop believin jou...
                                ...                        
273227                                 hey everybody follow
801045    next time think deepika padukone uses depressi...
801381    supporting mental illness limited depression a...
801491    emoji police cars revolving light tickets stil...
138185               venkateshkumar pleeessss help dont say
Name: Tweet, Length: 4690, dtype: object

In [33]:
vector = tfid.fit_transform(X)

In [12]:
#X = pd.DataFrame(X.toarray(),columns = tfid.get_feature_names_out())

In [34]:
simple_cross_val_recall = cross_val_score(nb, vector , y , scoring= 'recall', cv=3,
               n_jobs = -1 , verbose = 1
               )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.0s finished


In [35]:
print('the recall: ' ,np.mean(simple_cross_val_recall))

the recall:  0.15950652375853197


In [36]:
simple_cross_val_accuracy = cross_val_score(nb, vector , y , scoring= 'accuracy', cv=3,
               n_jobs = -1 , verbose = 1
               )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.6s finished


In [37]:
print('the accuracy: ' , np.mean(simple_cross_val_accuracy) )

the accuracy:  0.5554359689298402


## grid search

In [92]:
tfid2 = TfidfVectorizer(ngram_range=(4,5))
nb2 = MultinomialNB()
X = X.apply(str)
X

463236                                  bhunt get tacky day
801463    spent th grade crying absolutely reason wish s...
801367    added video youtube playlist http youtu kfxvmu...
800891    everyone war going inside happiness depression...
800290    friends struggle app helps avoid potentially h...
                                ...                        
59105                            holani nah overate da luau
800628                             welp im goign depression
105932    learned aobut thus quot movies han solo shot f...
157753    junkinthetrunkk great video amber looked like ...
801344    protects onset https goo gl xswslg amjpsychiat...
Name: Tweet, Length: 4690, dtype: object

In [93]:
pipe = Pipeline([
    ('TfidfVectorizer', tfid2),
    ('MultinomialNB()' , nb2)
    
])

In [94]:
pipe.get_params()

{'memory': None,
 'steps': [('TfidfVectorizer',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(4, 5), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words=None, strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=None, use_idf=True, vocabulary=None)),
  ('MultinomialNB()',
   MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
 'verbose': False,
 'TfidfVectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.float64'>, encoding='utf-8',
                 input='content', lowercase=True, max_df=1.0, max_features=None,
                 min_df=1, ngram_range=(4, 5), norm='l2', preprocessor=None,
              

In [95]:
pipe_grid = {
    'TfidfVectorizer__ngram_range': [(1,2) , (2,3), (3,4),(4, 5)],
    'MultinomialNB()__alpha': [0.1 , 0.5 , 1.0]
    
}

In [96]:
search_recall= GridSearchCV(pipe,
    pipe_grid,
    scoring='recall',
    n_jobs=-1,
    
   
    cv=3,
    verbose=1)

In [97]:
search_recall.fit(X , y )

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    4.8s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('TfidfVectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(4, 5),
                                               

In [98]:
print('the best params for recall: ',search_recall.best_params_)
print('the best recall score: ' , search_recall.best_score_)

the best params for recall:  {'MultinomialNB()__alpha': 1.0, 'TfidfVectorizer__ngram_range': (1, 2)}
the best recall score:  0.9773958016096268


In [104]:
best_model_recall = search_recall.best_estimator_


In [110]:
best_model_recall["MultinomialNB()"].params()



AttributeError: 'MultinomialNB' object has no attribute 'params'

In [99]:
search_accuracy= GridSearchCV(pipe,
    pipe_grid,
    scoring='accuracy',
    n_jobs=-1,
    
   
    cv=3,
    verbose=1)

In [100]:
search_accuracy.fit(X , y)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.1s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('TfidfVectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(4, 5),
                                               

In [101]:
print('the best params for accuracy: ',search_accuracy.best_params_)
print('the best accuracy score: ' , search_accuracy.best_score_)

the best params for accuracy:  {'MultinomialNB()__alpha': 1.0, 'TfidfVectorizer__ngram_range': (1, 2)}
the best accuracy score:  0.9057575574111255


# testeo y analisis del modelo

In [ ]:
'''{'MultinomialNB()__alpha': 1.0, 'TfidfVectorizer__ngram_range': (1, 2)}'''

In [142]:
tfid3 = TfidfVectorizer(ngram_range=(1,2))
nb3 = MultinomialNB(alpha = 1.0)

In [143]:
tfid3.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [144]:
vector3 = tfid3.transform(X)

In [145]:
tfid3.get_feature_names()

['100',
 '100 flashbacks',
 '11',
 '11 status',
 '111',
 '111 think',
 '15',
 '180425',
 '180425 mscpedit',
 '1xe',
 '1xe great',
 '2009',
 '2009 alright',
 '22',
 '22 jessleal7',
 '23275879',
 '2d3dilumdu',
 '2d3dilumdu barbara',
 '33',
 '33 kaotic2',
 '3eb0f4cc',
 '3eb0f4cc fd6',
 '4562',
 '4562 src',
 '53',
 '53 depression',
 '5adf8637e4b061c0bfa27b19',
 '5adf8637e4b061c0bfa27b19 ncid',
 '6cd2la93',
 '6ticzy',
 '6ticzy funniest',
 '76795',
 '76795 real',
 '83',
 '83 yeah',
 '86',
 '86 smrutimishra7',
 'a10066808',
 'a10066808 utm',
 'a1j3hy',
 'a1j3hy rekyewflst',
 'a1o',
 'a1o alot',
 'a2xa',
 'a690a0cebade',
 'a6ly',
 'a6yvb33h2y',
 'a6zft61se',
 'a8319686',
 'a8319686 html',
 'a9b8c66acf80fca040784e5e9cd',
 'aa',
 'aa poc',
 'aaa',
 'aaa shooters',
 'aaaaaaaand',
 'aaaaaaaand back',
 'aaaaaah',
 'aaaaaah https',
 'aaaaaand',
 'aaaaaand comes',
 'aaaaayyyyy',
 'aaaaayyyyy momma',
 'aaaah',
 'aaaah ok',
 'aaahhhh',
 'aaahhhh hypocrisy',
 'aac3a',
 'aakash',
 'aakash great',
 'aamir

In [146]:
nb3.fit(vector3, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [147]:
nb3.coef_

array([[-10.89164741, -10.89164741, -10.83601501, ..., -10.85551731,
        -11.03384898, -11.03384898]])

In [151]:
coefs = pd.Series(nb3.coef_[0], index = tfid3.get_feature_names())


In [152]:
coefs.sort_values()

#agregar palabras a las stopwords , por ejemplo 'twitter', 'com'
#agregar precision, accuracy, recall , f1 , etc
#min, max 
#max_features:overfitting? ~ regularizacion

#to do: traduccion?
#red de deeplearning para buscar mejor score
#testeos
#

zzzzzzzzzz love   -11.033849
retweet great     -11.033849
retweet hope      -11.033849
marathon woke     -11.033849
edit couple       -11.033849
                     ...    
twitter com        -7.341041
https              -7.334380
twitter            -7.328687
com                -7.174650
depression         -6.182616
Length: 50908, dtype: float64

/home/lucaspancotto/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'manic depression jimmy tough'

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/lucaspancotto/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'bhunt get tacky day'

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/lucaspancotto/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test 